In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree as skKNN
from cuML import KNN as cumlKNN
import pygdf
import os

# Helper Functions

In [2]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [3]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz',source='mortgage'):
    if os.path.exists(cached) and source=='mortgage':
        print('use mortgage data')
        with gzip.open(cached) as f:
            X = np.load(f)
        X = X[np.random.randint(0,X.shape[0]-1,nrows),:ncols]
    else:
        print('use random data')
        X = np.random.rand(nrows,ncols)
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])}).fillna(0)
    return df

In [4]:
def pd2pygdf(df):
    # convert pandas dataframe to pygdf dataframe
    if isinstance(df,np.ndarray):
        df = pd.DataFrame({'fea%d'%i:df[:,i] for i in range(df.shape[1])})
    pdf = pygdf.DataFrame()
    for c,column in enumerate(df):
        pdf[c] = df[column]
    return pdf

In [5]:
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=1e-2,with_sign=True,metric='mse'):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    if metric=='mse':
        error = mean_squared_error(a,b)
    else:
        error = np.sum(a!=b)/(a.shape[0]*a.shape[1])
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,pygdf.DataFrame) or isinstance(x,pygdf.Series):
        return x.to_pandas().values
    return x    

# Run tests

In [6]:
%%time
nrows = 2**16
ncols = 40

X = load_data(nrows,ncols)
print('data',X.shape)

use mortgage data
data (65536, 40)
CPU times: user 4.68 s, sys: 696 ms, total: 5.38 s
Wall time: 5.09 s


In [7]:
n_neighbors = 10

In [8]:
%%time
knn_sk = skKNN(X)
D_sk,I_sk = knn_sk.query(X,n_neighbors)

CPU times: user 3min 18s, sys: 820 ms, total: 3min 19s
Wall time: 3min 11s


In [9]:
%%time
X = pd2pygdf(X)

CPU times: user 6.88 s, sys: 1.04 s, total: 7.92 s
Wall time: 793 ms


In [10]:
%%time
knn_cuml = cumlKNN(n_gpus=1)
knn_cuml.fit(X)
D_cuml,I_cuml = knn_cuml.query(X,n_neighbors)

CPU times: user 2.28 s, sys: 348 ms, total: 2.62 s
Wall time: 2.71 s


In [11]:
passed = array_equal(D_sk,D_cuml)
message = 'compare knn: cuml vs sklearn distances %s'%('equal'if passed else 'NOT equal')
print(message)
passed = array_equal(I_sk,I_cuml)
message = 'compare knn: cuml vs sklearn indexes %s'%('equal'if passed else 'NOT equal')
print(message)

compare knn: cuml vs sklearn distances equal
compare knn: cuml vs sklearn indexes NOT equal
